 ## Week 5, Part 2 Assignment
 #### Group 4
 ##### Rough Draft

In [ ]:
import os
import nltk

spamfolder = '/Users/andrew/Documents/School/Web Analytics/HW4/enron1/spam'
spamdata = []
for filename in os.listdir(spamfolder):
    with open(spamfolder+'/'+filename) as spamtext:
        spamtext = spamtext.read()
        spamtext = spamtext.decode('UTF8', errors='ignore')
        spamdata.append(spamtext)
        #spamdata.append(spamtext.read())
        
hamfolder = '/Users/andrew/Documents/School/Web Analytics/HW4/enron1/ham'
hamdata = []
for filename in os.listdir(hamfolder):
    with open(hamfolder+'/'+filename) as hamtext:
        hamtext = hamtext.read()
        hamtext = hamtext.decode('UTF8', errors='ignore')
        hamdata.append(hamtext)

In [141]:
type(spamdata[1])

unicode

In [137]:
hamdata[1]

'Subject: vastar resources , inc .\r\ngary , production from the high island larger block a - 1 # 2 commenced on\r\nsaturday at 2 : 00 p . m . at about 6 , 500 gross . carlos expects between 9 , 500 and\r\n10 , 000 gross for tomorrow . vastar owns 68 % of the gross production .\r\ngeorge x 3 - 6992\r\n- - - - - - - - - - - - - - - - - - - - - - forwarded by george weissman / hou / ect on 12 / 13 / 99 10 : 16\r\nam - - - - - - - - - - - - - - - - - - - - - - - - - - -\r\ndaren j farmer\r\n12 / 10 / 99 10 : 38 am\r\nto : carlos j rodriguez / hou / ect @ ect\r\ncc : george weissman / hou / ect @ ect , melissa graves / hou / ect @ ect\r\nsubject : vastar resources , inc .\r\ncarlos ,\r\nplease call linda and get everything set up .\r\ni \' m going to estimate 4 , 500 coming up tomorrow , with a 2 , 000 increase each\r\nfollowing day based on my conversations with bill fischer at bmar .\r\nd .\r\n- - - - - - - - - - - - - - - - - - - - - - forwarded by daren j farmer / hou / ect on 12 / 10 

In [142]:
labeled_emails = ([(ham_mail.split(), 'ham') for ham_mail in hamdata] +
                  [(spam_mail.split(), 'spam') for spam_mail in spamdata])
import random
random.shuffle(labeled_emails)

all_emails = [email for email, classifcation in labeled_emails]

In [162]:
flattened = []
for email in all_emails:
    for word in email:
        flattened.append(word)

tokenized_emails = []
for word in flattened:
        tokenized_emails.extend(nltk.word_tokenize(word))

In [163]:
tokenized_emails[:30]

[u'Subject',
 u':',
 u'bounce',
 u'skel',
 u'@',
 u'iit',
 u'.',
 u'demokritos',
 u'.',
 u'gr',
 u':',
 u'non',
 u'-',
 u'member',
 u'submission',
 u'from',
 u'[',
 u'washington',
 u'.',
 u'mutual',
 u',',
 u'``',
 u'inc',
 u'.',
 u'``',
 u']',
 u'from',
 u'paliourg',
 u'@',
 u'iit']

In [168]:
word_freq = nltk.FreqDist(w for w in tokenized_emails)
top_words = list(word_freq)[:2000]

In [169]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in top_words:
        features['contains({})'.format(word)] = (word in document_words)
    return features

 ## Train Classifier

In [178]:
featuresets = [(document_features(d), c) for (d,c) in labeled_emails]
train_set, test_set = featuresets[200:], featuresets[:200]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [179]:
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)

0.85
Most Informative Features
       contains(effects) = True             spam : ham    =     39.5 : 1.0
    contains(affiliated) = True             spam : ham    =     37.9 : 1.0
    contains(aggressive) = True             spam : ham    =     37.9 : 1.0
         contains(boost) = True             spam : ham    =     34.7 : 1.0
       contains(watches) = True             spam : ham    =     28.2 : 1.0
